# First: load imports needed for the project and preparation of the project

In [2]:
# This cell is necessary to use this notebook in google colab
# If you are running this notebook in colab, please change colab to True

import os

colab = True
cwd = os.getcwd()

if colab is True and cwd != "/content/Bsc_Thesis":
  ! git clone https://github.com/SergioTallo/Bsc_Thesis.git
  % cd Bsc_Thesis

print(cwd)

fatal: destination path 'Bsc_Thesis' already exists and is not an empty directory.
/content/Bsc_Thesis
/content


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import math
from torch import Tensor, float32, sin, cos
import torch.nn as nn
import matplotlib.pyplot as plt
import utils_bsc
import statistics
from tqdm import tqdm

if torch.cuda.is_available():
  device = torch.device('cuda:0')
  print('Device: GPU =', torch.cuda.get_device_name(0))
else:
  device = torch.device('cpu')
  print('Device: CPU')


Device: GPU = Tesla T4


In [4]:
utils_bsc.print_versions()

versions of packages:
Python: 3.7.12
Pandas: 1.3.5
Numpy: 1.21.5
PyTorch: 1.10.0+cu111
Sklearn: 1.0.2


# Data loading and preparation

Now, we should create a dataset with all the data stored in the .csv file

Description of the data:

*   time: Timestamp (YYYY-MM-DD HH:MM:SS)
*   PLN1: Power in the phase 1 (W)
*   PLN2: Power in the phase 2 (W)
*   PLN3: Power in the phase 3 (W)
*   ULL1: Current Voltage between 2 phases (V)
*   ULL2: Current Voltage between 2 phases (V)
*   ULL3: Current Voltage between 2 phases (V)
*   COS_PHI1: Phase shift (Cos)
*   COS_PHI2: Phase shift (Cos)
*   COS_PHI3: Phase shift (Cos)
*   FREQ: Electricity Frequency (Hz)
*   RC_DC: Fault currents
*   RC_AC: Fault currents
*   RC_50Hz: Fault currents
*   RC_150Hz: Fault currents
*   RC_<100Hz: Fault currents
*   RC_100Hz-1kHz: Fault currents
*   RC_>10kHz: Fault currents


In [5]:
dataset = pd.read_csv('data_factory.csv')
dataset.head()

,time,PLN1,PLN2,PLN3,ULL1,ULL2,ULL3,COS_PHI1,COS_PHI2,COS_PHI3,FREQ,RC_DC,RC_AC,RC_50Hz,RC_150Hz,RC_<100Hz,RC_100Hz-1kHz,RC_>1kHz,RC_>10kHz
0,2020-06-01 00:00:00,1141.0819,519.5034,482.9381,398.8613,400.1982,395.6010,0.8091,0.6864,0.4875,49.9927,4.0,91.0,10.0,39.0,36.0,86.0,82.0,7.0
1,2020-06-01 00:01:00,1145.1162,519.1807,491.4436,398.6934,400.1579,395.5431,0.8080,0.6903,0.4904,49.9779,5.0,64.0,7.0,27.0,25.0,60.0,55.0,2.0
2,2020-06-01 00:02:00,1140.9558,743.3837,484.9942,398.4367,400.1205,395.5259,0.8113,0.9274,0.4806,49.9782,4.0,64.0,7.0,27.0,25.0,60.0,55.0,2.0
3,2020-06-01 00:03:00,1151.9409,741.4836,487.4224,398.9800,400.4375,395.8621,0.8249,0.9123,0.4778,49.9850,5.0,66.0,8.0,28.0,25.0,61.0,57.0,2.0
4,2020-06-01 00:04:00,1142.1594,741.9858,486.7629,398.7133,400.3145,395.6446,0.8081,0.9291,0.4552,49.9856,4.0,85.0,11.0,45.0,41.0,75.0,68.0,6.0


Once we have the dataset, we should prepare it. Finding the missing or the NaN values and replace them with suitable values (in this case we use the previous value).

In [6]:
# Replace all mising values with NaN
dataset = dataset.replace(' ', np.nan)
# Search for all the rows with NaN values
nan_values = dataset[dataset.isna().any(axis=1)]
# Print the shape to know how many are there
print(f'Number of rows with NaN values before cleaning: {nan_values.shape[0]}') 

# Fill all NaN values with the previous row value
dataset_clean = dataset.fillna(method='ffill')

# Check that there isn't any NaN values
nan_values = dataset_clean[dataset_clean.isna().any(axis=1)]
# Print the shape to know how many are there
print(f'Number of rows with NaN values after cleaning: {nan_values.shape[0]}') 

#Total number of samples
print(f'Total number of samples: {dataset_clean.shape[0]}')
print(f'Number of features: {dataset_clean.shape[1]}')

Number of rows with NaN values before cleaning: 2546
Number of rows with NaN values after cleaning: 0
Total number of samples: 63360
Number of features: 19


# Distribution of the data

Now we look at the distribution of the different features of the data over different time intervals.
First we take a look of the min and max values, mean and median value and the standard deviation of every feature.

In [7]:
for column in dataset_clean.columns:
  if column == 'time':
    print(column)
    print('Min value: ', dataset_clean[column].min())
    print('Max value: ', dataset_clean[column].max())
    print('')
  else:
    print(column)
    print('Min value: ', dataset_clean[column].min())
    print('Max value: ', dataset_clean[column].max())
    print('Mean value: ', dataset_clean[column].mean())
    print('Median value: ', dataset_clean[column].median())
    print('Standard deviation: ', dataset_clean[column].std())
    print('')

time
Min value:  2020-06-01 00:00:00
Max value:  2020-07-14 23:59:00

PLN1
Min value:  1136.6154
Max value:  26071.959
Mean value:  7185.2709603472795
Median value:  4370.43185
Standard deviation:  5619.401583329123

PLN2
Min value:  330.2979
Max value:  16792.3887
Mean value:  4645.443629026148
Median value:  4504.5791
Standard deviation:  3948.189347661856

PLN3
Min value:  465.1976
Max value:  14512.8389
Mean value:  4081.8074168496946
Median value:  3310.39745
Standard deviation:  3423.479296987532

ULL1
Min value:  384.115
Max value:  418.8481
Mean value:  398.5690919760104
Median value:  399.0723
Standard deviation:  4.60327571313802

ULL2
Min value:  386.556
Max value:  420.7076
Mean value:  400.6287229008857
Median value:  401.09614999999997
Standard deviation:  4.379149353374775

ULL3
Min value:  384.4315
Max value:  418.0959
Mean value:  398.1920267992438
Median value:  398.7023
Standard deviation:  4.1864513722862595

COS_PHI1
Min value:  0.791
Max value:  1.0
Mean value:  0

In [8]:
# Set to True to print the graphs
print_graphs = False

if print_graphs is True:

  for i, column in enumerate(dataset_clean.columns):
    if i > 0:
      # Feature in a weekly interval
      utils_bsc.week_plot(dataset_clean, i, column)
      # Feature in a daily interval (only the values of weekdays between 4:00 and 19:30)
      utils_bsc.daily_plot(dataset_clean, i, column)

# Dataset normalisation

We divide every valu by the max value of this specific feature to have a value between 0 and 1

In [9]:
# apply the mean / stddev scaling in Pandas using the .mean() and .std() methods
def normalize_mean_std_dataset(df):
    # copy the dataframe
    df_scaled = df.copy()
    # apply mean / stddev scaling
    for column in tqdm(df_scaled.columns):
      if column != 'time':
        df_scaled[column] = (df[column] - df[column].mean()) / df[column].std()
    return df_scaled

dataset_norm = normalize_mean_std_dataset(dataset_clean)

dataset_norm.head()

100%|██████████| 19/19 [00:00<00:00, 870.92it/s]


,time,PLN1,PLN2,PLN3,ULL1,ULL2,ULL3,COS_PHI1,COS_PHI2,COS_PHI3,FREQ,RC_DC,RC_AC,RC_50Hz,RC_150Hz,RC_<100Hz,RC_100Hz-1kHz,RC_>1kHz,RC_>10kHz
0,2020-06-01 00:00:00,-1.075593,-1.045021,-1.051232,0.063478,-0.098312,-0.618908,-1.868350,-1.835847,-1.500292,-0.345935,-0.817380,0.632551,1.075812,0.995360,1.143832,0.694697,0.747095,2.141318
1,2020-06-01 00:01:00,-1.074875,-1.045103,-1.048747,0.027004,-0.107515,-0.632738,-1.884005,-1.803753,-1.486828,-1.139728,0.678985,-0.849829,-0.918340,-0.792166,-0.630653,-0.822036,-0.777047,-1.175568
2,2020-06-01 00:02:00,-1.075615,-0.988316,-1.050631,-0.028760,-0.116055,-0.636846,-1.837041,0.147415,-1.532327,-1.123638,-0.817380,-0.849829,-0.918340,-0.792166,-0.630653,-0.822036,-0.777047,-1.175568
3,2020-06-01 00:03:00,-1.073661,-0.988798,-1.049922,0.089264,-0.043667,-0.556540,-1.643493,0.023152,-1.545327,-0.758922,0.678985,-0.740023,-0.253623,-0.643206,-0.630653,-0.763700,-0.664147,-1.175568
4,2020-06-01 00:04:00,-1.075401,-0.988670,-1.050114,0.031327,-0.071754,-0.608493,-1.882582,0.161405,-1.650254,-0.726741,-0.817380,0.303134,1.740530,1.889123,1.950416,0.053002,-0.043201,1.477941


# Preparation Training and Test set

Once the dataset is prepared, make batches of data,put them togheter in an array and split them into train and test sets.
After looking through the dataset and the features, i decided to takeonly the values with a timestap of a weekday between 4:00 and 19:30. In many of the features in the interval otside those timestamps there i only noise, which can be a sign that the machine is off in that time interval.

In [10]:
# Create 27840 batches of longitud 60

endset = utils_bsc.create_batches(dataset=dataset_norm, batch_size=30, device=device)
    
print(f'\n\n{len(endset)} sequences of longitud {endset[0].shape[0]} with {endset[0].shape[1]} features')

# Spliting into train and test sets

training_data, testing_data = train_test_split(endset, test_size=0.1, random_state=25)

train_set = torch.stack(training_data).float().to(device)
test_set = torch.stack(testing_data).float().to(device)

print(f'length of training set: {train_set.shape[0]}')
print(f'length of test set: {test_set.shape[0]}')

100%|██████████| 32/32 [00:16<00:00,  1.99it/s]



28800 sequences of longitud 30 with 18 features
length of training set: 25920
length of test set: 2880


# Create a Baseline Model

I am taking the median of all features to create an statistical baselinemodel. I will use this baseline model to compare the results of the actual model with it. Everything that works better than this baseline model could be an improvement.

!!!!Last step as prediction!!!!

In [11]:
criterion = nn.MSELoss()

losses = []

for i in range(1, len(train_set)):
  loss = criterion(train_set[i], train_set[i - 1])
  losses.append(float(loss))

print("Mean Loss of baselinemodel: ", np.mean(losses))
print("Standard deviation Loss of baselinemodel: ", np.std(losses))


Mean Loss of baselinemodel:  0.9703539133329461
Standard deviation Loss of baselinemodel:  0.5991845827967649


In [1]:
class ANN_relu(nn.Module):

    def __init__(self, D_in, D_out, device):
        super(ANN_relu, self).__init__()
        self.linear1 = nn.Linear(D_in, 512).to(device)
        self.linear2 = nn.Linear(512, 1024).to(device)
        self.linear3 = nn.Linear(1024, 512).to(device)
        self.linear4 = nn.Linear(512, D_out).to(device)

    def forward(self, x):
        x = self.linear1(x)
        x = torch.relu(x)
        x = self.linear2(x)
        x = torch.relu(x)
        x = self.linear3(x)
        x = torch.relu(x)
        x = self.linear4(x)
        
        return x

# This function trains the model for one epoch
# Nothing to do here
def train(model, device, train_set, optimizer, n_epochs):
    model.train()

    for e in range(1, n_epochs +1):

      for batch_idx in tqdm(range(len(train_set))):

        batch = train_set[batch_idx]

        losses = []

        for i in range(len(batch) - 1):
          data, target = batch[i].to(device), batch[i + 1].to(device)
          optimizer.zero_grad()
          output = model(data)
          loss = criterion(output, target)
          losses.append(float(loss))
          loss.backward()
          optimizer.step()
      
      print(f'\nEpoch {e}:')
      
      print("\nMean Loss of Model FFN: ", np.mean(losses))

model_FFN = ANN_relu(18, 18, device)

n_epochs = 10
lr=0.01

# Define Loss, Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_FFN.parameters(), lr=lr)

train(model_FFN, device, train_set, optimizer, n_epochs)

def train2(model, device, train_set, optimizer, n_epochs):
    model.train()

    for e in range(1, n_epochs +1):

      for batch_idx in tqdm(range(len(train_set))):

        batch = train_set[batch_idx]

        losses = []
        optimizer.zero_grad()
        
        for i in range(len(batch) - 1):
          data, target = batch[i].to(device), batch[i + 1].to(device)

          output = model(data)
          loss = criterion(output, target)
          losses.append(float(loss))
          loss.backward()
          optimizer.step()
      
      print(f'\nEpoch {e}:')
      
      print("\nMean Loss of Model FFN: ", np.mean(losses))

model_FFN = ANN_relu(18, 18, device)

n_epochs = 10
lr=0.01

# Define Loss, Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_FFN.parameters(), lr=lr)

train2(model_FFN, device, train_set, optimizer, n_epochs)

NameError: ignored

  3%|▎         | 836/25920 [00:39<20:02, 20.86it/s]

# RNN Model

We train before a standard RNN and see which results we can expected with a small and easy solution.
I am using this template (https://github.com/gabrielloye/RNN-walkthrough/blob/master/main.ipynb) and make changes using it as a base.

1) torch.nn.RNN (https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)

Parameters
* input_size – The number of expected features in the input x
* hidden_size – The number of features in the hidden state h
* num_layers – Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two RNNs together to form a stacked RNN, with the second RNN taking in outputs of the first RNN and computing the final results. Default: 1
* nonlinearity – The non-linearity to use. Can be either 'tanh' or 'relu'. Default: 'tanh'
* bias – If False, then the layer does not use bias weights b_ih and b_hh. Default: True
* batch_first – If True, then the input and output tensors are provided as (batch, seq, feature) instead of (seq, batch, feature). Note that this does not apply to hidden or cell states. See the Inputs/Outputs sections below for details. Default: False
* dropout – If non-zero, introduces a Dropout layer on the outputs of each RNN layer except the last layer, with dropout probability equal to dropout. Default: 0
* bidirectional – If True, becomes a bidirectional RNN. Default: False

2) torch.nn.Linear (https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)

Parameters
* in_features – size of each input sample
* out_features – size of each output sample
* bias – If set to False, the layer will not learn an additive bias. Default: True


In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, device, batch_first = True, dropout = 0):
        super(RNN, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.device = device

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size = input_size, hidden_size = hidden_dim, num_layers = n_layers, batch_first = batch_first, dropout = dropout)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        # Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size).to(self.device)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        # We'll send the tensor holding the hidden state to the device we specified earlier as well
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

In [ ]:
# Instantiate the model with hyperparameters

model_rnn = RNN(input_size = 18,
                output_size = 18,
                hidden_dim = 36,
                n_layers = 1,
                batch_first = True,
                dropout = 0,
                device = device)

# We'll also set the model to the device that we defined earlier (default is CPU)
model_rnn = model_rnn.to(device)

# Define hyperparameters
n_epochs = 10
lr=0.01

# Define Loss, Optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_rnn.parameters(), lr=lr)

print(train_set.size(0))

losses = []

print_ = True

# Training Run

for epoch in range(1, n_epochs + 1):

  optimizer.zero_grad() # Clears existing gradients from previous epoch

  for i in tqdm(range(len(train_set) -1)):

    output, hidden = model_rnn(train_set[i].unsqueeze(0))

    if print_ is True:
      print('')
      print(output[0])
      print(train_set[i + 1][0])
      print_ = False

    loss = criterion(output, train_set[i + 1])
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly

  print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
  print("Loss: {:.4f}".format(loss.item()))

In [ ]:
# Show results of the loss function

fig = plt.figure()

ax = fig.add_subplot(111)
plt.ion()

fig.show()
fig.canvas.draw()

ax.plot(losses)
ax.set_title("Mean Squared Error")
fig.canvas.draw()

# Transformer Model settings

Now, we define a class with the transformer model that we are going to use:

Using the already written pytorch library for Transformers:

1) torch.nn.TransformerEncoderLayer (https://pytorch.org/docs/stable/generated/torch.nn.TransformerEncoder.html)

*   d_model –> the number of expected features in the input (required).
*   nhead –> the number of heads in the multiheadattention models (required).
*   dropout –> the dropout value (default=0.1).
*   activation –> the activation function of the intermediate layer, can be a string (“relu” or “gelu”) or a unary callable. (default: relu)
*   layer_norm_eps –> the eps value in layer normalization components (default=1e-5).
*   batch_first –> If True, then the input and output tensors are provided as (batch, seq, feature). (default: False)
*   norm_first –> if True, layer norm is done prior to attention and feedforward operations, respectivaly. Otherwise it’s done after. (default: False (after))

2) torch.nn.TransformerDecoderLayer

* d_model –> the number of expected features in the input (required).
* nhead –> the number of heads in the multiheadattention models (required).
* dim_feedforward –> the dimension of the feedforward network model (default=2048).
* dropout –> the dropout value (default=0.1).
* activation –> the activation function of the intermediate layer, can be a string (“relu” or “gelu”) or a unary callable. Default: relu
* layer_norm_eps –> the eps value in layer normalization components (default=1e-5).
* batch_first –> If True, then the input and output tensors are provided as (batch, seq, feature). Default: False.
* norm_first –> if True, layer norm is done prior to self attention, multihead attention and feedforward operations, respectivaly. Otherwise it’s done after. Default: False (after).

3) torch.nn.TransformerEncoder

* encoder_layer –> an instance of the TransformerEncoderLayer() class (required).
* num_layers –> the number of sub-encoder-layers in the encoder (required).
* norm –> the layer normalization component (optional).


4) torch.nn.TransformerDecoder

* decoder_layer – an instance of the TransformerDecoderLayer() class (required).
* num_layers – the number of sub-decoder-layers in the decoder (required).
* norm – the layer normalization component (optional).


In [ ]:
def positional_encoding(seq_len: int, dim_model: int, device):
    
    # Tensor with the positions of every sequence element (0 to seq_len)
    pos = torch.arange(seq_len, dtype=float32, device=device).reshape(1, -1, 1)
    
    # Tensor with the positions of every feature in the sequence (0 to dim_model)
    dim = torch.arange(dim_model, dtype=float32, device=device).reshape(1, 1, -1)

    phase = pos / (1e4 ** (torch.div(dim, dim_model, rounding_mode='floor')))

    position_encoding = torch.where(dim.long() % 2 == 0, sin(phase), cos(phase))

    return position_encoding.to(device)


class Transformer(nn.Module):
    def __init__(self, feature_size, output_size, num_encoder_layers, num_heads, num_decoder_layers, device, debug: bool = False, dropout: float =0.1, batch_first: bool = False):
        super(Transformer, self).__init__()
        
        encoder_layer = nn.TransformerEncoderLayer(d_model= feature_size, nhead= num_heads, dropout=dropout, device=device, batch_first=batch_first)
        decoder_layer = nn.TransformerDecoderLayer(d_model= feature_size, nhead= num_heads, dropout=dropout, device=device, batch_first=batch_first)
        
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers= num_encoder_layers)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers= num_decoder_layers)

        self.output_size = output_size
        self.device = device
        self.debug = debug

    def generate_square_mask(self, dim):
        return torch.triu(torch.ones(dim, dim) * float('-inf'), diagonal=1).to(self.device)
        
    def forward (self, src):
        
        mask = self.generate_square_mask(len(src))
        if self.debug is True:
            print('mask creation: ok')

        src_pos = src + positional_encoding(src.shape[1], src.shape[2], self.device)
        if self.debug is True:
            print('Pos encoder (encoder): ok')

        output = self.encoder (src, mask)
        if self.debug is True:
            print('encoder pass: ok')
        
        output = self.decoder (src_pos, output, mask)
        if self.debug is True:
            print('decoder pass: ok')
        return output

We should define an optimizer too.
For this, we use the pytorch library:

* SGD –> Stochastic gradient descent.

1) torch.optim.SDG (https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD)

* params (iterable) – iterable of parameters to optimize or dicts defining parameter groups
* lr (float) – learning rate
* momentum (float, optional) – momentum factor (default: 0)
* weight_decay (float, optional) – weight decay (L2 penalty) (default: 0)
* dampening (float, optional) – dampening for momentum (default: 0)
* nesterov (bool, optional) – enables Nesterov momentum (default: False)

In [ ]:
# Initialize Transformer Model and Optimizer

model = Transformer (num_encoder_layers=3,
                     num_decoder_layers=3,
                     feature_size=18,
                     output_size=18,
                     num_heads=3,
                     device = device,
                     batch_first=False,
                     debug=False)

print('Setup model Ok')

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

print('Setup optimizer Ok')

train_transformer = False

if train_transformer is True:

  # List to keep track of the losses
  loss_list = []
  # Initialize optimizer gradients
  optimizer.zero_grad()

  epochs = 1

  out = []

  for e in range(epochs):
    print('Epoch: ', e + 1)

    for i in tqdm(range(len(train_set))):

        batch = train_set[i].unsqueeze(0)
        net_out = model.forward(batch)
        out.append(net_out)
        loss = torch.mean((net_out - batch) ** 2).to(device)

        loss.backward()
        optimizer.step()

        loss_list.append(loss.item())


In [ ]:
# Show results of the loss function

fig = plt.figure()

ax = fig.add_subplot(111)
plt.ion()

fig.show()
fig.canvas.draw()

ax.plot(loss_list)
ax.set_title("Mean Squared Error")
fig.canvas.draw()

Ideas, things to remember, to search, etc...

reconstruction, vergelich mit base line model